In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from binance.client import Client
import ta
from sklearn.preprocessing import MinMaxScaler
import joblib

# Autenticación en la API de Binance
api_key = 'T4Bw573BSJCbaHscSo8jn37lE1SOoGsNircF8f7B061WIKBxNkP5nx68vvAev9uk'
api_secret = 'p7VN6HHKR6ZtGkfEeZC3FongJPkR2yr7AjPtPDC8IeaOkASdKZYCKyTuiIfGw57q'
client = Client(api_key, api_secret)

# Obtención de los datos históricos de precios de Bitcoin
klines = client.futures_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "1 year ago UTC")

# Creación de un dataframe con los datos
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                     'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 
                                     'taker_buy_quote_asset_volume', 'ignore'])

# Eliminación de las columnas que no se van a utilizar
data = data.drop(['timestamp', 'close_time', 'quote_asset_volume', 'number_of_trades', 
                  'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)

# Conversión de los precios a float
data['open'] = data['open'].astype(float)
data['high'] = data['high'].astype(float)
data['low'] = data['low'].astype(float)
data['close'] = data['close'].astype(float)
data['volume'] = data['volume'].astype(float)

# Calcula los indicadores técnicos
data['momentum'] = ta.momentum.roc(data['close'], fillna=True)
data['ATR'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'], fillna=True)
data['ADX'] = ta.trend.adx(data['high'], data['low'], data['close'], fillna=True)
data['MACD'] = ta.trend.macd_diff(data['close'], fillna=True)

# Crear y ajustar el scaler
scalers = {}
for column in ['open', 'high', 'low', 'close', 'momentum', 'ATR', 'ADX', 'MACD']:
    scaler = MinMaxScaler()
    data[column] = scaler.fit_transform(data[[column]])
    scalers[column] = scaler
    # Guardar el scaler para uso posterior
    joblib.dump(scaler, f'scalers/{column}_scaler.gz')

# Procesar los datos para la entrada del modelo
data['next_close'] = data['close'].shift(-1)
data.dropna(inplace=True)

X = []
y = []

n = 48  # Número de pasos de tiempo en la secuencia

# Añade los indicadores técnicos a la entrada del modelo
for i in range(n, len(data)):
    X.append(data[['close', 'momentum', 'ATR', 'ADX', 'MACD']].iloc[i-n:i].values)
    y.append(data['next_close'].values[i])

# Convertir a numpy arrays
X, y = np.array(X), np.array(y)

# Redimensionar X a la forma adecuada para la entrada LSTM [muestras, pasos de tiempo, características]
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)
])

# Compilar el modelo
model.compile(loss='mse', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, batch_size=20, validation_data=(X_val, y_val))

# Guardar el modelo entrenado
model.save('modelos de entrenamiento/btc_price_prediction_model_Indicators_copy.h5')


2023-05-17 16:30:32.403903: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-17 16:30:32.429297: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-17 16:30:32.430282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 16:30:32.908559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:785: RuntimeW

Epoch 1/200


2023-05-17 16:30:51.610557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-17 16:30:51.611628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-17 16:30:51.612532: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

349/349 [==============================] - ETA: 0s - loss: 0.0041

2023-05-17 16:31:01.380368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-17 16:31:01.381488: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-17 16:31:01.382408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

349/349 [==============================] - 11s 25ms/step - loss: 0.0041 - val_loss: 5.1407e-04
Epoch 2/200
349/349 [==============================] - 8s 24ms/step - loss: 3.5808e-04 - val_loss: 4.1665e-04
Epoch 3/200
349/349 [==============================] - 8s 24ms/step - loss: 3.4569e-04 - val_loss: 3.0715e-04
Epoch 4/200
349/349 [==============================] - 8s 24ms/step - loss: 3.3755e-04 - val_loss: 3.6394e-04
Epoch 5/200
349/349 [==============================] - 8s 24ms/step - loss: 3.3188e-04 - val_loss: 3.6865e-04
Epoch 6/200
349/349 [==============================] - 8s 24ms/step - loss: 2.9926e-04 - val_loss: 3.7733e-04
Epoch 7/200
349/349 [==============================] - 9s 24ms/step - loss: 2.9372e-04 - val_loss: 3.1554e-04
Epoch 8/200
349/349 [==============================] - 8s 24ms/step - loss: 2.6429e-04 - val_loss: 2.2311e-04
Epoch 9/200
349/349 [==============================] - 8s 24ms/step - loss: 2.4208e-04 - val_loss: 2.8393e-04
Epoch 10/200
349/349 [===